In [2]:
import pandas as pd

### Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [5]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
# Получаем максимум, минимум и кол-во timestamp для каждого пользователя
users = ratings.groupby('userId').agg(['min', 'max', 'count'])['timestamp'].reset_index()
# Для тех у кого оценок больше 100 вычисляем разницу между максимумом и минимумом по времени
users = users[users['count']>100].assign(diff=users['max']-users['min'])
users.sort_values('diff', ascending=False)
users['diff'].mean()

44731367.73469388

### Задание 2

Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов.

Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [91]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [92]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [93]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [94]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [95]:
#Делаем JOIN таблиц, взяв только client_id от client_base
table1 = pd.merge(client_base['client_id'], rzd, how='left').fillna(0)
table1 = pd.merge(table1, auto, how='left').fillna(0)
table1 = pd.merge(table1, air, how='left').fillna(0)
table1.head()

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0


In [96]:
# Здесь просто объединяем все таблицы
table2 = client_base.merge(rzd, how='left').merge(auto, how='left').merge(air, how='left').fillna(0)
table2.head()

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,0.0,0.0
1,112,Энтузиастов 8а,2810.0,0.0,0.0
2,113,Левобережная 1а,10283.0,57483.0,0.0
3,114,Мира 14,5774.0,83.0,0.0
4,115,ЗЖБИиДК 1,981.0,912.0,81.0


### Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:

- У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
- Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
- Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

In [87]:
# Сделаем тестовую таблицу
analytics = pd.DataFrame(
    {'user_id': [123,234,342,535,362], 
     'orders': [2525,5624,2674,2625,2636],
     'visits': [16825, 1952, 21890, 376, 19509],
     'date': ['2020-01-01 01:01:01', '2020-02-01 01:01:01', '2020-03-01 01:01:01', 
              '2020-04-01 01:01:01', '2020-05-01 01:01:01']
    }
)

# Таблица с пользователями и координатами при заказе
orders = pd.DataFrame(
    {'user_id': [234,342,362], 
     'coord': [['55.742793','37.615401'],['54.742793','33.615401'],['52.742793','34.615401']]
    }
)

# 1. Для связывания с покупкой нужно время визита или покупки, тогда при наименьшей разнице по времени с таблицей
#    координат (ищем разницу по времени между всеми координатами и покупками в течении дня)
#    эти данные можно будет связать
# 2. Нам будут нужны координаты, наиболее близкие по времени ко времени совершения покупки
#    (выберем наименьшую разницу)
# 3. Можно узнать насколько удаленно человек передвигается для совершения покупок, а также в какие дни
#    например в будние дни он совершает покупки только в определенном месте около дома, а в выходные может уезжать
#    за город или в удаленный гипермаркет, на основе этого можно решить когда и где человек делает покупки